In [1]:
from pymle.models import CIR
from pymle.sim.Simulator1D import Simulator1D
from pymle.TransitionDensity import ExactDensity, KesslerDensity
from pymle.fit.AnalyticalMLE import AnalyticalMLE
import numpy as np
import pickle

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import torch

In [5]:
SP = pd.read_csv('/Users/hby/Downloads/CNNpred/Processed_S&P.csv')

In [9]:
SP['Date'][1]

'2010-01-04'

In [6]:
SP_Sub = SP[['Date', 'Close', 'Volume', 'mom1', 'mom3', 'ROC_15', 'EMA_20', 'Oil', 'Gold', 'DAAA', 'AAPL', 
             'JNJ', 'XOM', 'TE1', 'DE1', 'S&P-F', 'Dollar index-F']]

In [7]:
SP_Sub = SP_Sub.iloc[19:, :].reset_index(drop=True)

In [8]:
SP_Sub

,Date,Close,Volume,mom1,mom3,ROC_15,EMA_20,Oil,Gold,DAAA,AAPL,JNJ,XOM,TE1,DE1,S&P-F,Dollar index-F
0,2010-01-29,1073.869995,-0.007254,-0.011818,-0.004203,-5.940312,1123.157501,-0.010459,-0.008732,5.24,-0.036279,-0.014734,-0.008159,3.61,0.96,-0.81,0.70
1,2010-02-01,1089.189941,-0.246680,-0.009829,0.004880,-4.872578,1119.922495,0.021414,0.007418,5.30,0.013902,0.003659,0.027161,3.63,0.95,1.47,-0.29
2,2010-02-02,1103.319946,0.164785,0.014266,-0.011818,-3.806521,1118.341300,0.037629,0.022549,5.28,0.005803,0.014265,0.011786,3.63,0.95,1.01,-0.30
3,2010-02-03,1097.280029,-0.097713,0.012973,-0.009829,-3.427148,1116.335465,-0.003238,0.003825,5.36,0.017206,-0.005626,-0.005376,3.68,0.95,-0.07,0.47
4,2010-02-04,1063.109985,0.367345,-0.005474,0.014266,-7.207079,1111.266372,-0.049766,-0.028693,5.27,-0.036039,-0.015559,-0.028228,3.58,0.97,-3.17,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,2017-11-09,2584.620117,-0.017375,0.001444,0.001271,0.878967,2571.803178,0.005984,0.000623,3.60,-0.002043,-0.006864,0.005990,1.28,0.66,-0.27,-0.44
1961,2017-11-10,2582.300049,-0.089962,-0.003762,-0.000189,0.275321,2572.802880,-0.007173,-0.000389,0.00,-0.006880,-0.005629,-0.012266,1.37,0.00,-0.17,-0.07
1962,2017-11-13,2584.840088,-0.024084,-0.000898,0.001444,0.774279,2573.949281,0.000352,-0.004944,3.69,-0.004008,0.001433,-0.000603,1.36,0.64,0.10,0.12
1963,2017-11-14,2578.870117,0.070184,0.000984,-0.003762,0.379126,2574.417932,-0.019376,-0.002621,3.66,-0.015118,-0.001932,-0.007842,1.34,0.65,-0.15,-0.71


In [ ]:
SP_Sub.to_csv('SP_Sub.csv', header=True, index=False)

In [7]:
SP_Sub.isnull().sum()

Date              0
Close             0
Volume            0
mom1              0
mom3              0
ROC_15            0
EMA_20            0
Oil               0
Gold              0
DAAA              0
AAPL              0
JNJ               0
XOM               0
TE1               0
DE1               0
S&P-F             0
Dollar index-F    0
dtype: int64

In [8]:
SP_Sub['Close'] /= 1000
SP_Sub['Volume'] *= 10
SP_Sub['mom1'] *= 10
SP_Sub['mom3'] = SP_Sub['mom3']*10/3
SP_Sub['ROC_15'] = SP_Sub['ROC_15']/15*10
SP_Sub['EMA_20'] = SP_Sub['EMA_20']/1000/20
SP_Sub['Oil'] *= 10
SP_Sub['Gold'] *= 10
SP_Sub['DAAA'] /= 10
SP_Sub['AAPL'] *= 10
SP_Sub['JNJ'] *= 10
SP_Sub['XOM'] *= 10
SP_Sub['TE1'] /= 10

In [14]:
SP_Sub.iloc[:, 1:].to_numpy().shape

(1965, 16)

In [ ]:
def m_a(a, n=10) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
with open('cot_sam_dety_reg.pickle', 'rb') as fp:
    sam_dy = pickle.load(fp)

with open('cot_var_dety_reg.pickle', 'rb') as fp:
    var_dy = pickle.load(fp)
    
with open('cot_sam_randy_reg.pickle', 'rb') as fp:
    sam_ry = pickle.load(fp)

with open('cot_var_randy_reg.pickle', 'rb') as fp:
    var_ry = pickle.load(fp)

In [ ]:
with open('ot_sam_dety_reg.pickle', 'rb') as fp:
    sam_otdy = pickle.load(fp)

with open('ot_var_dety_reg.pickle', 'rb') as fp:
    var_otdy = pickle.load(fp)
    
with open('ot_sam_randy_reg.pickle', 'rb') as fp:
    sam_otry = pickle.load(fp)

with open('ot_var_randy_reg.pickle', 'rb') as fp:
    var_otry = pickle.load(fp)

In [ ]:
with open('ot_cost_randy_reg.pickle', 'rb') as fp:
    cost_otry = pickle.load(fp)

In [ ]:
cost_otry[:].mean()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(m_a(np.array(cost_otry.detach()), n=50)[700:] - 1.0, label='Cost')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(m_a(np.array(var_dy - sam_dy), n=20)[20:], label='Det y')
plt.plot(m_a(np.array(var_ry - sam_ry), n=20)[20:], label='Rand y')
plt.plot(m_a(np.array(var_otdy - sam_otdy), n=20)[20:], label='OT Det y')
plt.plot(m_a(np.array(var_otry - sam_otry), n=20)[20:], label='OT Rand y')
# plt.plot(m_a(np.array(dual_ot - sam_ot), n=10)[20:], label='OT')
plt.legend(loc='best')
plt.show()

In [ ]:
var_dy[-50:].mean() - sam_dy[-50:].mean(), var_otdy[-50:].mean() - sam_otdy[-50:].mean()

In [ ]:
var_ry[-50:].mean() - sam_ry[-50:].mean(), var_otry[-50:].mean() - sam_otry[-50:].mean()

In [ ]:
# ===========================
# Set the true model (CIR) params, to simulate the process
# ===========================
model = CIR()  # Cox-Ingersol-Ross 

kappa = 3  # rate of mean reversion
mu = 0.3  # long term level of process
sigma = 0.2  # volatility

model.params = np.array([kappa, mu, sigma])

# ===========================
# Simulate a sample path (we will fit to this path)
# ===========================
S0 = 0.4  # initial value of process
T = 1  # num years of the sample
freq = 100  # observations per year
dt = 1. / freq
seed = 123  # random seed: set to None to get new results each time

simulator = Simulator1D(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
sample = simulator.sim_path()

# ===========================
# Fit maximum Likelihood estimators
# ===========================
# Set the parameter bounds for fitting  (kappa, mu, sigma)
param_bounds = [(0, 10), (0, 4), (0.01, 1)]

# Choose some initial guess for params fit
guess = np.array([1, 0.1, 0.4])

# Fit using Kessler MLE
kessler_est = AnalyticalMLE(sample, param_bounds, dt, density=KesslerDensity(model)).estimate_params(guess)

print(f'\nKessler MLE: {kessler_est} \n')

# Fit using Exact MLE
exact_est = AnalyticalMLE(sample, param_bounds, dt, density=ExactDensity(model)).estimate_params(guess)

print(f'\nExact MLE: {exact_est}')

In [ ]:
sample.shape